In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 4s (61.9 kB/s)
Reading 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...|2015-08-31 00:00:00|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...| 

In [4]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_table = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3DT59XH7HXR9K|          5|            0|          0|   N|                Y|
|R1LFS11BNASSU8|          5|            0|          1|   N|                Y|
|R296RT05AG0AF6|          5|            0|          0|   N|                Y|
|R3V37XDZ7ZCI3L|          5|            0|          1|   N|                Y|
|R14GU232NQFYX2|          5|            0|          0|   N|                Y|
| RZQH4V7L2O1PL|          1|            1|          1|   N|                Y|
|R1F8JMOSPJ3KO7|          5|            0|          0|   N|                Y|
|R1ZISGY2BWW4Z5|          5|            0|          0|   N|                Y|
|R17PW4I3AE5WZW|          5|            0|          0|   N|                Y|
|R3D93G1KTP6A8P|          3|            0|          0|   N|     

In [5]:
# The data is filtered to create a DataFrame or table where there are 20 or more total votes

total_votes_20_df = vine_table.filter('total_votes >= 20')
total_votes_20_df.count()

107421

In [6]:
# The data is filtered to create a DataFrame or table where there is a Vine review, total_votes is equal to or greater than 50%

import pyspark.sql.functions as F

helpful_total_df = total_votes_20_df.withColumn('help_perct', F.round((total_votes_20_df.helpful_votes/total_votes_20_df.total_votes)*100,2))
cleaned_help_perct_df = helpful_total_df.filter('help_perct >= 50')
cleaned_help_perct_df.count()

99046

In [7]:
# The data is filtered to create a DataFrame or table where there is a Vine review

paid_vine_df = cleaned_help_perct_df.filter("vine =='Y'")
paid_vine_df.count()

1207

In [8]:
# The data is filtered to create a DataFrame or table where there isn’t a Vine review

unpaid_vine_df = cleaned_help_perct_df.filter("vine =='N'")
unpaid_vine_df.count()

97839

In [9]:
# The total number of reviews, the number of 5-star reviews, and the percentage 5-star reviews are calculated for all Vine and non-Vine reviews 

total_5_star = cleaned_help_perct_df.filter('star_rating == 5').count()
total_5_star

46367

In [10]:
total_1_star = cleaned_help_perct_df.filter('star_rating == 1').count()
total_1_star

22561

In [11]:
total_reviews = cleaned_help_perct_df.count()
total_reviews

99046

In [12]:
total_below5 = cleaned_help_perct_df.filter('star_rating < 5').count()
total_below5

52679

In [15]:
total_5_star_paid = paid_vine_df.filter('star_rating == 5').count()
print(f'1. Percent of vine 5 star reviews: {total_5_star_paid}')

total_5_star_unpaid = unpaid_vine_df.filter('star_rating == 5').count()
print(f'2. Percent of non_vine 5 star reviews: {total_5_star_unpaid}')

total_below5_paid = paid_vine_df.filter('star_rating < 5').count()
print(f'3. Percent of vine below 5 star reviews: {total_below5_paid}')

total_below5_unpaid = unpaid_vine_df.filter('star_rating < 5').count()
print(f'4. Percent of nonvine below 5 star reviews: {total_below5_unpaid}')

1. Percent of vine 5 star reviews: 509
2. Percent of non_vine 5 star reviews: 45858
3. Percent of vine below 5 star reviews: 698
4. Percent of nonvine below 5 star reviews: 51981


In [16]:
perct_1 = round((total_1_star / total_reviews)*100,2)
print(f'1. Percent of total 1 star: {perct_1}%')


perct_5 = round((total_5_star / total_reviews)*100,2)
print(f'2. Percent of total 5 star: {perct_5}%')


perct_5_paid = round((total_5_star_paid / paid_vine_df.count())*100,2)
print(f'3. Percent of Vine reviews that were 5 star: {perct_5_paid}%')


perct_5_unpaid = round((total_5_star_unpaid / unpaid_vine_df.count())*100,2)
print(f'4. Percent of non-Vine reviews that were 5 star: {perct_5_unpaid}%')


perct_below5_paid = round((total_below5_paid / paid_vine_df.count())*100,2)
print(f'5. Percent of Vine reviews that below 5 star: {perct_below5_paid}%')


perct_below5_unpaid = round((total_below5_unpaid / unpaid_vine_df.count())*100,2)
print(f'6. Percent of non-Vine reviews that below 5 star: {perct_below5_unpaid}%')

1. Percent of total 1 star: 22.78%
2. Percent of total 5 star: 46.81%
3. Percent of Vine reviews that were 5 star: 42.17%
4. Percent of non-Vine reviews that were 5 star: 46.87%
5. Percent of Vine reviews that below 5 star: 57.83%
6. Percent of non-Vine reviews that below 5 star: 53.13%
